**Download Yelp Dataset from Kaggle**

In [ ]:
 # install kaggle
 ! pip install -q kaggle

In [ ]:
# Download kaggle.json file from your kaggle profile and upload
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arpandas65","key":"f8a8961daac3b7c0ed74367f0202c90d"}'}

In [ ]:
# Download yelp dataset from kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.92G/4.92G [02:06<00:00, 56.3MB/s]
100% 4.92G/4.92G [02:06<00:00, 41.8MB/s]


In [ ]:
!pip3 install sentence_transformers
!pip3 install transformers

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 2.1MB 4.2MB/s 
     |████████████████████████████████| 1.2MB 26.5MB/s 
     |████████████████████████████████| 870kB 35.5MB/s 
     |████████████████████████████████| 3.3MB 38.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=d7863f36cf4fe502864a09c34a4e1a9945d8a8419342ddd9cd64089e7600340c
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=44496e308e3f37af148967b3833d5d23f5f886242c8fe407de4140eea81b1fad
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:
# UnZip the downloaded data
import zipfile
with zipfile.ZipFile('/content/yelp-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [ ]:
# Remove the ZIP file
import os
os.remove("/content/yelp-dataset.zip")

**Data Extraction From Yelp**

In [ ]:
# Download pyspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 75kB/s 
     |████████████████████████████████| 204kB 39.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6aef474d8e7e22dbfe517225c543bc97d42e173edeb5bd41a38d615e4ca4028c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.appName('yelp_dataset').getOrCreate()

In [ ]:
reviews_sk = spark.read.json('/content/yelp_academic_dataset_review.json')

**Filtering Review Samples and Coverting to Pandas Dataframe**

In [ ]:
# We will get a sample of 10L from the dataset
sample_review=reviews_sk.sample(False, 0.1, seed=0).limit(1000000)
reviews_pd = sample_review.select("*").toPandas()

In [ ]:
# Overview of the converted sample
reviews_pd.to_csv('yelp_review_sample.csv')
reviews_pd.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,_sS2LBIGNT5NQb6PD1Vtjw,0,2013-05-28 20:38:06,0,NDhkzczKjLshODbqDoNLSg,5.0,I called WVM on the recommendation of a couple...,0,eC5evKn1TWDyHCyQAwguUw
1,EEHhKSxUvJkoPSzeGKkpVg,0,2014-05-07 18:10:21,0,KKVFopqzcVfcubIBxmIjVA,5.0,I work in the Pru and this is the most afforda...,0,99RsBrARhhx60UnAC4yDoA
2,6Hm2FmfLcU_M91TrZI5htA,3,2014-02-05 21:09:05,3,LBJJWJ0uNlIybMfPnQGS0A,5.0,I loved everything about this place. I've only...,5,bUHweiErUJ36WGeNrPmEbA
3,_gOz7-aHMyGUHOtjDrEv2w,0,2017-09-20 15:38:27,2,WxrK52MRJy6Cxk_vOJeD1Q,2.0,"When we first arrived at the Coco Key, we were...",4,9d22a-5XXvsZ55p04gI5jA
4,EuiU19EjhybpGLXqBzntgw,0,2012-04-28 11:39:17,0,gfwRzTeqcgzyKvFnFwU3TQ,1.0,Bummer. I was very disappointed. So I finally...,4,Sy_D4Tt_XJXSb03UigFZTA


In [ ]:
# Save data to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp 'yelp_review_sample.csv' '/content/drive/MyDrive' 

In [ ]:
!cp 'yelp_business.csv' '/content/drive/MyDrive' 

**Extracting Business Data**

We will extract business information and then filter the listings that serves foods

In [ ]:
business_sk=spark.read.json('/content/yelp_academic_dataset_business.json')
business_pd=business_sk.select("*").toPandas()

In [ ]:
business_pd.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

As we are interested only in restaurents/ places that serves food by any means we will filter the businesses using the category column 

In [ ]:
list(business_pd.categories.unique())

['Gastropubs, Food, Beer Gardens, Restaurants, Bars, American (Traditional), Beer Bar, Nightlife, Breweries',
 'Salad, Soup, Sandwiches, Delis, Restaurants, Cafes, Vegetarian',
 'Antiques, Fashion, Used, Vintage & Consignment, Shopping, Furniture Stores, Home & Garden',
 'Beauty & Spas, Hair Salons',
 'Gyms, Active Life, Interval Training Gyms, Fitness & Instruction',
 'Restaurants, Thai',
 'Dentists, Health & Medical, Orthodontists',
 'Breakfast & Brunch, Restaurants',
 'Shopping, Jewelry Repair, Appraisal Services, Local Services, Jewelry, Engraving, Gold Buyers',
 'Barbers, Beauty & Spas',
 'Food, Shopping, Convenience Stores, Souvenir Shops',
 'Wigs, Hair Extensions, Hair Salons, Blow Dry/Out Services, Hair Stylists, Beauty & Spas, Shopping',
 'Food, Pizza, Restaurants',
 'Restaurants, American (New), Bakeries, Desserts, Food, Cupcakes',
 'Books, Mags, Music & Video, Musical Instruments & Teachers, Pawn Shops, Shopping, Videos & Video Game Rental, Jewelry',
 'Oil Change Stations, S

In [ ]:
# If we look at the categories, all most every store selling food has one of these tags
food_tags=['Food','Restaurants','Cafes']

In [ ]:
# Save the dataset
business_pd.to_csv('yelp_business.csv')

In [ ]:
# We will add a column serves_food to identify if the store serves food or not
serves_food=[]
for i in range(len(business_pd)):
    try:
        if(business_pd.iloc[i]['categories'] is not None):
            categories=business_pd.iloc[i]['categories'].split(',')
            categories=[en.replace(' ','') for en in categories]
            matched_tags=(list(set(categories).intersection(food_tags)))
            if(len(matched_tags)>0):
                serves_food.append(1)
            else:
                serves_food.append(0) 
        else:
            serves_food.append(0)             
    except:
        print(business_pd.iloc[i]['categories'])
        serves_food.append(2)                  

In [ ]:
business_pd.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,serves_food
0,921 Pearl St,"(None, None, 'beer_and_wine', {'touristy': Fal...",6iYb2HFDywm3zjuRg0shjw,"Gastropubs, Food, Beer Gardens, Restaurants, B...",Boulder,"(11:0-23:0, 11:0-23:0, 11:0-23:0, 11:0-23:0, 1...",1,40.017544,-105.283348,Oskar Blues Taproom,80302,86,4.0,CO,1
1,7000 NE Airport Way,"(None, None, u'beer_and_wine', {'romantic': Fa...",tCbdrRPZA0oiIYSmHG3J0w,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Portland,"(5:0-18:0, 5:0-18:0, 5:0-18:0, 5:0-18:0, 5:0-1...",1,45.588906,-122.593331,Flying Elephants at PDX,97218,126,4.0,OR,1
2,4720 Hawthorne Ave,"(None, None, None, None, None, None, None, Fal...",bvN78flM8NLprQ1a1y5dRg,"Antiques, Fashion, Used, Vintage & Consignment...",Portland,"(11:0-18:0, None, 11:0-18:0, 11:0-18:0, 11:0-1...",1,45.511907,-122.613693,The Reclaimory,97214,13,4.5,OR,0
3,2566 Enterprise Rd,"(None, None, None, None, None, None, None, Non...",oaepsyvc0J17qwi8cfrOWg,"Beauty & Spas, Hair Salons",Orange City,None,1,28.914482,-81.295979,Great Clips,32763,8,3.0,FL,0
4,1046 Memorial Dr SE,"(None, None, None, None, None, None, None, Non...",PE9uqAjdw0E4-8mjGl3wVA,"Gyms, Active Life, Interval Training Gyms, Fit...",Atlanta,"(16:0-19:0, 16:0-19:0, 9:0-11:0, None, 16:0-19...",1,33.747027,-84.353424,Crossfit Terminus,30316,14,4.0,GA,0


In [ ]:
business_pd['serves_food']=serves_food
business_food=business_pd[business_pd['serves_food']==1]
business_food.to_csv('yelp_business_food.csv')

In [ ]:
# Copy the dataset to drive
!cp 'yelp_business_food.csv' '/content/drive/MyDrive' 

In [ ]:
# Overview of the transfomed dataset
business_food.shape

(63849, 15)

In [ ]:
business_food.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,serves_food
0,921 Pearl St,"(None, None, 'beer_and_wine', {'touristy': Fal...",6iYb2HFDywm3zjuRg0shjw,"Gastropubs, Food, Beer Gardens, Restaurants, B...",Boulder,"(11:0-23:0, 11:0-23:0, 11:0-23:0, 11:0-23:0, 1...",1,40.017544,-105.283348,Oskar Blues Taproom,80302,86,4.0,CO,1
1,7000 NE Airport Way,"(None, None, u'beer_and_wine', {'romantic': Fa...",tCbdrRPZA0oiIYSmHG3J0w,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Portland,"(5:0-18:0, 5:0-18:0, 5:0-18:0, 5:0-18:0, 5:0-1...",1,45.588906,-122.593331,Flying Elephants at PDX,97218,126,4.0,OR,1


**Extracting Reviews of Food Stores**

In [ ]:
# ids of the food stores
business_food_ids=business_food['business_id']

In [ ]:
# From the review dataset we will extract the reviews for food stores
business_food_reviews=reviews_pd[reviews_pd['business_id'].isin(business_food_ids)]
business_food_reviews.to_csv('yelp_business_food_reviews.csv')
business_food_reviews.shape

(609652, 9)

In [ ]:
!cp 'yelp_business_food_reviews.csv' '/content/drive/MyDrive' 

In [ ]:
# Food store Reviews
business_food_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
1,EEHhKSxUvJkoPSzeGKkpVg,0,2014-05-07 18:10:21,0,KKVFopqzcVfcubIBxmIjVA,5.0,I work in the Pru and this is the most afforda...,0,99RsBrARhhx60UnAC4yDoA
2,6Hm2FmfLcU_M91TrZI5htA,3,2014-02-05 21:09:05,3,LBJJWJ0uNlIybMfPnQGS0A,5.0,I loved everything about this place. I've only...,5,bUHweiErUJ36WGeNrPmEbA
4,EuiU19EjhybpGLXqBzntgw,0,2012-04-28 11:39:17,0,gfwRzTeqcgzyKvFnFwU3TQ,1.0,Bummer. I was very disappointed. So I finally...,4,Sy_D4Tt_XJXSb03UigFZTA
5,wVlxPWwFL-MrzCXim-UfWg,0,2018-05-08 17:14:19,0,DS4kN9ph17evKTUu0I3heQ,5.0,We had lunch there on a Sunday afternoon and i...,0,1hrlsEQ8l43jK4aXqQknGg
9,JCgLWYjoPJlznzFEtnzqKg,1,2013-03-18 18:06:09,0,CK03ZpXOZjfJOEHFRL9TLg,2.0,Nice staff. Subpar food for the price.,1,wmJJs3LePX6KJwgMwUm2tQ


In [ ]:
# Overview of the dataset
print('Number of unique Reviews:',str(len(business_food_reviews)))
print('Number of unique Businesses:',str(len(business_food_reviews.business_id.unique())))
print('Number of unique Users that posted a Review:',str(len(business_food_reviews.user_id.unique())))

Number of unique Reviews: 609652
Number of unique Businesses: 55134
Number of unique Users that posted a Review: 362513


In [ ]:
# Merging the store info with review  dataset
review_business_data_merged=pd.merge(business_food_reviews,business_food,how='left',on='business_id')

In [ ]:
# Copy to dive
review_business_data_merged.to_csv('yelp_reviews_business_merged.csv')
!cp 'yelp_reviews_business_merged.csv' '/content/drive/MyDrive' 

In [ ]:
# Drop unwanted columns
review_business_data_merged=review_business_data_merged.drop(columns=['hours','attributes','useful','funny'])
review_business_data_merged.head()

,business_id,cool,date,review_id,stars_x,text,user_id,address,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars_y,state,serves_food
0,EEHhKSxUvJkoPSzeGKkpVg,0,2014-05-07 18:10:21,KKVFopqzcVfcubIBxmIjVA,5.0,I work in the Pru and this is the most afforda...,99RsBrARhhx60UnAC4yDoA,"Prudential Center Food Court, 800 Boylston St","Sandwiches, Seafood, Soup, Restaurants, Fast Food",Boston,0,42.348484,-71.082457,Boston Chowda Co,02199,110,3.5,MA,1
1,6Hm2FmfLcU_M91TrZI5htA,3,2014-02-05 21:09:05,LBJJWJ0uNlIybMfPnQGS0A,5.0,I loved everything about this place. I've only...,bUHweiErUJ36WGeNrPmEbA,1305 Commercial Drive,"Restaurants, Canadian (New), Greek, Bars, Loun...",Vancouver,0,49.272800,-123.069730,The Storm Crow Tavern,V5L 3X5,225,3.5,BC,1
2,EuiU19EjhybpGLXqBzntgw,0,2012-04-28 11:39:17,gfwRzTeqcgzyKvFnFwU3TQ,1.0,Bummer. I was very disappointed. So I finally...,Sy_D4Tt_XJXSb03UigFZTA,75 Summer St,"Seafood, Restaurants, Food",Arlington,1,42.419266,-71.156433,Fresh Pond Seafood Market & Takeout,02474,96,4.0,MA,1
3,wVlxPWwFL-MrzCXim-UfWg,0,2018-05-08 17:14:19,DS4kN9ph17evKTUu0I3heQ,5.0,We had lunch there on a Sunday afternoon and i...,1hrlsEQ8l43jK4aXqQknGg,288 Derby St,"Italian, Restaurants, Pizza",Salem,1,42.520415,-70.891657,Bambolina Restaurant,01970,360,4.5,MA,1
4,JCgLWYjoPJlznzFEtnzqKg,1,2013-03-18 18:06:09,CK03ZpXOZjfJOEHFRL9TLg,2.0,Nice staff. Subpar food for the price.,wmJJs3LePX6KJwgMwUm2tQ,1403 Lavaca St,"Restaurants, Cafes",Austin,0,30.276880,-97.742046,Capitol Grounds Cafe,78701,9,4.0,TX,1


**Read Dataset**

In [ ]:
review_business_data_merged=pd.read_csv('/content/drive/MyDrive/yelp_reviews_business_merged.csv')
review_business_data_merged=review_business_data_merged.drop(columns=['Unnamed: 0','cool','funny','attributes','is_open','serves_food','useful','hours'])